In [1]:
!pip install ultralytics
!pip install roboflow
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [5]:
from roboflow import Roboflow
import os
import shutil

rf = Roboflow(api_key="TZCi2SCltMQkbaVAtOqT")

# List of datasets to download (workspace, dataset, version)
datasets_info = [
    ("hongmo", "sea-ezx3q", 44),
    ("kim-borma", "marine-debris-vb2qm", 16),
    ("myfyp", "new-taco", 3),
    ("anas-fd5bu", "-kg", 5),
"""
    ("trash-detection-eka7g", "aeriel-trash-detection", 5),
    ("fyp-bfx3h", "yolov8-trash-detections", 6),
    ("taco-ihjgk", "yolov8-trash-detections-kgnug", 11),
    ("trash-ai-27vay", "ultimate-rqkdd", 1),
    ("iqram", "trashdetectionv8", 2),
    ("smart-india-hackathon-2023", "garbage_best", 1),
    # New datasets (replace VERSION with the correct version number)
    ("kim-borma", "conttttt", 1),
    ("adamson-university-rkm6w", "manila-bay", 3),
    ("davids-workspace", "garbage-detection-mnj17", 1),
    ("majasworkspace", "p_eine_klasse", 1),
    ("ukm-wcn", "ml2-wcn-ukm", 23),
    ("vit-akped", "tacodetecron-2", 1),
    ("k2k", "imported-f38bi", 4),
    ("python-project-5oizd", "trash-detection-mzljs", 7),
    ("project-mqoot", "plastic-bag-detection", 1),
    ("trash-dataset-for-oriented-bounded-box", "trash-detection-1fjjc", 14),
    ("nam-nhat", "trash-dvdrr", 5),
    ("litter-beach-detection", "beach-garbage", 2),
    ("dechets-qlhbp", "tacoo-cj6rf", 1),
    ("cscsi218", "garbageclassification-47d7t", 1),
    ("hust-xz9js", "trashbot-wahhb", 1),
    ("furqan-sayyed-veuct", "taco-ppujr", 1),
    ("abhijeet-beedikar-pbc0x", "garbage-classification-taco", 1)
"""
]

dest_dir = os.path.abspath("datasets")
print(f"Destination directory: {dest_dir}")

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

for workspace, dataset_name, version in datasets_info:
    print(f"Processing {dataset_name}...")
    try:
        project = rf.workspace(workspace).project(dataset_name)
        print(f"Attempting to download version {version} of {dataset_name}")

        # Specify the download location
        download_dir = os.path.join(dest_dir, dataset_name)

        # Download the dataset to the specified location
        dataset = project.version(version).download("yolov11", location=download_dir)
        print(f"Successfully downloaded {dataset_name} to {dataset.location}")

        # List the contents of the downloaded directory
        if os.path.exists(dataset.location):
            print(f"Contents of {dataset.location}: {os.listdir(dataset.location)}")
        else:
            print(f"Warning: Downloaded directory not found at {dataset.location}")

    except Exception as e:
        print(f"Error processing {dataset_name}: {str(e)}")
    print("-" * 50)

print("Download process completed. Please check the following directory for your files:")
print(dest_dir)

Destination directory: /content/datasets
Processing sea-ezx3q...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 44 of sea-ezx3q
Successfully downloaded sea-ezx3q to /content/datasets/sea-ezx3q
Contents of /content/datasets/sea-ezx3q: ['test', 'README.dataset.txt', 'valid', 'data.yaml', 'train', 'README.roboflow.txt']
--------------------------------------------------
Processing marine-debris-vb2qm...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 16 of marine-debris-vb2qm
Successfully downloaded marine-debris-vb2qm to /content/datasets/marine-debris-vb2qm
Contents of /content/datasets/marine-debris-vb2qm: ['test', 'README.dataset.txt', 'valid', 'data.yaml', 'train', 'README.roboflow.txt']
--------------------------------------------------
Processing new-taco...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 3 of new-taco
Successfully downloaded new-taco to /cont

ValueError: too many values to unpack (expected 3)

In [6]:
import os
import shutil
import yaml
import glob

def merge_datasets(datasets_dir, combined_dataset_dir):
    # Create directories for the combined dataset
    os.makedirs(combined_dataset_dir, exist_ok=True)
    splits = ['train', 'valid', 'test']
    for split in splits:
        os.makedirs(os.path.join(combined_dataset_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(combined_dataset_dir, 'labels', split), exist_ok=True)

    combined_classes = []
    class_mappings = {}

    # First pass: Collect all class names and build combined_classes list
    for dataset_name in os.listdir(datasets_dir):
        dataset_path = os.path.join(datasets_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        data_yaml_path = os.path.join(dataset_path, 'data.yaml')
        if not os.path.exists(data_yaml_path):
            print(f"data.yaml not found in {dataset_path}. Skipping this dataset.")
            continue

        with open(data_yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)

        dataset_classes = data_yaml.get('names', [])
        if not dataset_classes:
            print(f"No 'names' field found in {data_yaml_path}. Skipping this dataset.")
            continue

        # Update combined_classes and create class mappings for this dataset
        mapping = {}
        for idx, class_name in enumerate(dataset_classes):
            if class_name not in combined_classes:
                combined_classes.append(class_name)
            mapping[idx] = combined_classes.index(class_name)
        class_mappings[dataset_name] = mapping

    print(f"Combined classes: {combined_classes}")

    # Second pass: Copy images and labels, remap class IDs
    for dataset_name in os.listdir(datasets_dir):
        dataset_path = os.path.join(datasets_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        mapping = class_mappings.get(dataset_name)
        if mapping is None:
            continue  # Skip datasets that were not included in combined_classes

        for split in ['train', 'valid', 'test']:
            split_dir = os.path.join(dataset_path, split)
            if not os.path.exists(split_dir):
                print(f"Split '{split}' not found in {dataset_name}. Skipping.")
                continue

            # Check if images and labels are under 'images' and 'labels' directories
            images_dir = os.path.join(split_dir, 'images')
            labels_dir = os.path.join(split_dir, 'labels')
            if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
                # Maybe images and labels are directly under split_dir
                images_dir = split_dir
                labels_dir = split_dir
                # Check if there are images and labels in split_dir
                image_files = glob.glob(os.path.join(images_dir, '*.jpg')) + \
                              glob.glob(os.path.join(images_dir, '*.jpeg')) + \
                              glob.glob(os.path.join(images_dir, '*.png')) + \
                              glob.glob(os.path.join(images_dir, '*.bmp')) + \
                              glob.glob(os.path.join(images_dir, '*.tif')) + \
                              glob.glob(os.path.join(images_dir, '*.tiff'))
                label_files = glob.glob(os.path.join(labels_dir, '*.txt'))
                if len(image_files) == 0:
                    print(f"No images found in {images_dir}. Skipping split '{split}' in dataset '{dataset_name}'.")
                    continue
            else:
                # Images and labels are under 'images' and 'labels' directories
                image_files = glob.glob(os.path.join(images_dir, '*.*'))
                label_files = glob.glob(os.path.join(labels_dir, '*.txt'))

            for image_file in image_files:
                image_filename = os.path.basename(image_file)
                image_name, image_ext = os.path.splitext(image_filename)

                # Corresponding label file
                label_file = os.path.join(labels_dir, f"{image_name}.txt")
                if not os.path.exists(label_file):
                    print(f"Label file {label_file} not found. Creating empty label.")
                    label_lines = []
                else:
                    with open(label_file, 'r') as lf:
                        label_lines = lf.readlines()

                # Remap class IDs in label file
                remapped_labels = []
                for line in label_lines:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        print(f"Invalid label format in {label_file}. Skipping line.")
                        continue
                    try:
                        class_id = int(parts[0])
                        new_class_id = mapping.get(class_id)
                        if new_class_id is None:
                            print(f"Class ID {class_id} not found in mapping for {dataset_name}. Skipping line.")
                            continue
                        parts[0] = str(new_class_id)
                        remapped_labels.append(' '.join(parts))
                    except ValueError:
                        print(f"Invalid class ID in {label_file}. Skipping line.")
                        continue

                # Copy image
                dest_image_name = f"{dataset_name}_{image_filename}"
                target_split = split  # Use the same split name
                dest_image_path = os.path.join(combined_dataset_dir, 'images', target_split, dest_image_name)
                shutil.copy(image_file, dest_image_path)

                # Write remapped label
                dest_label_name = f"{dataset_name}_{image_name}.txt"
                dest_label_path = os.path.join(combined_dataset_dir, 'labels', target_split, dest_label_name)
                with open(dest_label_path, 'w') as lf:
                    lf.write('\n'.join(remapped_labels))

    # Create combined data.yaml
    combined_data = {
        'path': os.path.abspath(combined_dataset_dir),
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'valid'),  # Use 'valid' as validation set
        'names': combined_classes
    }
    with open(os.path.join(combined_dataset_dir, 'data.yaml'), 'w') as f:
        yaml.dump(combined_data, f)

    print("Dataset merging complete.")

# Usage
datasets_dir = 'datasets'  # Directory containing individual datasets
combined_dataset_dir = 'combined_dataset'  # Directory to store the combined dataset

merge_datasets(datasets_dir, combined_dataset_dir)


Combined classes: ['Aluminium foil', 'Bottle', 'Bottle cap', 'Can', 'Carton', 'Cigarette', 'Glass bottle', 'Lid', 'Metal', 'Metal cap', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag', 'Plastic buoy', 'Plastic vessels', 'Pop tab', 'Straw', 'Styrofoam cup', 'Styrofoam piece', 'Styrofoam_Buoy', 'Wrapper', 'Glass', 'Net', 'PET_Bottle', 'Plastic_Buoy', 'Plastic_Buoy_China', 'Plastic_ETC', 'Rope', 'Styrofoam_Box', 'Styrofoam_Piece', 'PET', 'Styrofoam', 'cardboard', 'fabrics', 'glass', 'metal', 'other', 'plastic', 'wood']
Dataset merging complete.


In [ ]:
from ultralytics import YOLO
import os

import wandb
from wandb.integration.ultralytics import add_wandb_callback


# Set wandb project and run names
wandb_project_name = 'Edutech'

wandb.init(
    project=wandb_project_name,
    config={
        'epochs': 5,
        'batch_size': 16,
        'learning_rate': 0.001,
        # Add other hyperparameters if needed
    }
)


# Load the YOLO11 extra-large detection model
model = YOLO('yolo11x.pt')  # Detection model for object detection tasks

# Add WandB callback for logging
add_wandb_callback(model)


# Path to the combined dataset's data.yaml
data_yaml_path = os.path.join('combined_dataset', 'data.yaml')

# Verify that data.yaml exists
if not os.path.exists(data_yaml_path):
    raise FileNotFoundError(f"Error: data.yaml not found at {data_yaml_path}")
else:
    print(f"Using data.yaml at {data_yaml_path}")

# Train the model with wandb logging enabled
model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=16,
    # name=wandb_run_name,
    exist_ok=True,
    device='0',
    project=wandb_project_name,
    verbose=True,
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

train: WARNING ⚠️ /content/combined_dataset/images/train/-kg_frame_3069_patch_0_0_jpg.rf.11338fd82fcb4c2d2976c7d82b39339d.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/combined_dataset/images/train/-kg_renamed_DJI_0471_tile_3_7_jpg.rf.987a8f6df1d8eb7780053f92d712e45e.jpg: 1 duplicate labels removed


train: New cache created: /content/combined_dataset/labels/train.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 12132, len(boxes) = 110005. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
val: Scanning /content/combined_dataset/labels/valid... 6258 images, 135 backgrounds, 0 corrupt: 100%|██████████| 6258/6258 [00:05<00:00, 1056.34it/s]


val: New cache created: /content/combined_dataset/labels/valid.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 3174, len(boxes) = 21792. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to Edutech/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000227, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to Edutech/train
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      16.6G      1.374      4.093      1.416         87        640:  11%|█         | 215/2006 [02:45<22:51,  1.31it/s]

In [ ]:
# Run inference using the best model
results = model.predict(
    source=os.path.join(combined_dataset_dir, 'images/val'),
    save=True,
    project='runs/predict',
    name='yolo11x_predict',
    exist_ok=True
)


In [ ]:

import matplotlib.pyplot as plt
import glob
import os

def visualize_predictions(result_dir):
    image_paths = glob.glob(os.path.join(result_dir, '*.jpg'))
    num_images = min(4, len(image_paths))

    plt.figure(figsize=(15, 12))
    for i, image_path in enumerate(image_paths[:num_images]):
        image = plt.imread(image_path)
        plt.subplot(2, 2, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_predictions('runs/predict/yolo11x_predict')
